In [11]:
!pip install -r ../requirements.txt

In [51]:
import os
import json
import requests

import sqlite3
from tqdm.notebook import tqdm, trange
tqdm.pandas()


import numpy as np
import pandas as pd 
from sqlalchemy import create_engine

%reload_ext sql
%config SqlMagic.autocommit=True


from tqdm.notebook import tqdm
from pprint import pprint as print

In [52]:
# Make a database to keep all my lovely tables in
%sql sqlite:///../Data/Clean/gymternet.db --alias gymternet
engine = create_engine('sqlite:///../Data/Clean/gymternet.db')

Connecting to 'gymternet'

In [33]:
teams_df = pd.read_pickle('../Data/Raw/dirty_dfs/teams_df.pkl')
meets_df = pd.read_pickle('../Data/Raw/dirty_dfs/meets_df.pkl')
gymnasts_data_df = pd.read_pickle('../Data/Raw/dirty_dfs/gymnasts_data_df.pkl')
team_results_data_df = pd.read_pickle('../Data/Raw/dirty_dfs/team_results_data_df.pkl')

# Clean up `teams_df` and create `Teams.db`

What I want in this table in my database is:

| **team_id** | **team_name** | **team_url**                           |
|-------------|---------------|----------------------------------------|
| 1           | 'Auburn'      | "https://www.roadtonationals/team/645" | 


etc.

The `team_id` column should act as the primary key.


In [34]:
# Preview the dataframe
#teams_df.dtypes
teams_df.head()

,team_name,team_id,year,team_url
0,LSU,34,2024,https://www.roadtonationals.com/api/women/dash...
1,California,15,2024,https://www.roadtonationals.com/api/women/dash...
2,Utah,69,2024,https://www.roadtonationals.com/api/women/dash...
3,Florida,22,2024,https://www.roadtonationals.com/api/women/dash...
4,Stanford,61,2024,https://www.roadtonationals.com/api/women/dash...


This dataframe is already pretty tidy, but the links are pointing towards the API, which is not what we want, and we still have an irrelevant column (`year`)

In [35]:
base_team_url = 'https://roadtonationals.com/results/teams/dashboard'
teams_df['team_url'] = teams_df.apply(lambda x: f'{str(base_team_url)}/{str(x["year"])}/{str(x["team_id"])}', axis=1)

In [36]:
# Preview the df
teams_df.head()

,team_name,team_id,year,team_url
0,LSU,34,2024,https://roadtonationals.com/results/teams/dash...
1,California,15,2024,https://roadtonationals.com/results/teams/dash...
2,Utah,69,2024,https://roadtonationals.com/results/teams/dash...
3,Florida,22,2024,https://roadtonationals.com/results/teams/dash...
4,Stanford,61,2024,https://roadtonationals.com/results/teams/dash...


In [37]:
# Drop the year column
teams_df.drop(columns='year', inplace=True)
teams_df.head()

,team_name,team_id,team_url
0,LSU,34,https://roadtonationals.com/results/teams/dash...
1,California,15,https://roadtonationals.com/results/teams/dash...
2,Utah,69,https://roadtonationals.com/results/teams/dash...
3,Florida,22,https://roadtonationals.com/results/teams/dash...
4,Stanford,61,https://roadtonationals.com/results/teams/dash...


In [58]:
teams_df.to_sql('teams', con=engine, if_exists = 'append', chunksize = 1000)

89

In [30]:
%%sql --alias gymternet

CREATE TABLE teams 
(
    team_id INTEGER PRIMARY KEY,
    team_name TEXT,
    team_url TEXT
);

Running query in 'gymternet'

++
||
++
++

In [57]:
%%sql --alias gymternet
DROP TABLE teams;

Running query in 'gymternet'

++
||
++
++